In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("./word2vec_wrangling.csv")
exercise_to_loop = df["exercise_name"].to_list()

In [3]:
# -*- coding: utf-8 -*-
import re
from konlpy.tag import Mecab, Okt
from collections import Counter
import pandas as pd
import numpy as np

def preprocessing_hangul(text):
    # 개행문자 제거
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result


In [4]:
useless_list = ['점핑', '동작', '광주', '가능', '사진', '데일리', '공연', '하루', '카페', '영상', '바디', '일산', '검도', '살사', '대구', '사람', '볼링', '동요', '조깅', '등산', '체형', '클럽', '피지', '휘트', '무료', '자이로', '드럼', '발레', '수업', '폴린', '자격증', '피트', '트램', '홍대', '등록', '준비', '수영장', '센터', '댄스', '패들', '토닉', '요가', '일상', '클라이밍', '필라테스', '회원', '잠실', '여행', '취미', '여자', '킥복싱', '번지', '후프', '학원', '교육', '니스', '사랑', '부산', '스튜디오', '피티', '선생', '플라이', '레슨', '남자', '선수', '강사', '친구', '자세', '서핑', '교정', '클래스', '무용', '행복', '그룹', '수련', '트레이닝', '방송', '플라잉', '바이크', '수영', '키즈', '동호회', '지도자', '오늘', '타요', '개인', '헬스', '할인', '에어로빅', '기부', '상담', '라틴', '시작', '맛집', '아쿠아', '다이어트', '파운드', '카톡', '건강', '오픈', '마음', '수중', '탕가', '감사', '진행', '운동', '복싱', '소통', '문의', '전화', '사이클', '서프', '야사', '크로스', '근력', '기구', '시간', '핫요가', '피닝', '과정', '스포츠', '보드']
useful_list = ['점핑', '검도', '살사', '볼링', '조깅', '등산', '체형', '발레', '수영장', '센터', '토닉', '요가' '클라이밍', '필라테스', '취미', '킥복싱', '번지', '후프', '스튜디오', '피티', '플라이', '레슨', '자세', '서핑', '교정', '무용', '수련', '트레이닝', '플라잉', '바이크', '수영', '동호회', '개인', '헬스', '할인', '에어로빅', '아쿠아', '다이어트', '수중', '복싱', '사이클', '크로스', '기구', '핫요가', '보드']
useless_list = [item for item in useless_list if item not in useful_list]
print(useless_list)

['동작', '광주', '가능', '사진', '데일리', '공연', '하루', '카페', '영상', '바디', '일산', '대구', '사람', '동요', '클럽', '피지', '휘트', '무료', '자이로', '드럼', '수업', '폴린', '자격증', '피트', '트램', '홍대', '등록', '준비', '댄스', '패들', '요가', '일상', '클라이밍', '회원', '잠실', '여행', '여자', '학원', '교육', '니스', '사랑', '부산', '선생', '남자', '선수', '강사', '친구', '클래스', '행복', '그룹', '방송', '키즈', '지도자', '오늘', '타요', '기부', '상담', '라틴', '시작', '맛집', '파운드', '카톡', '건강', '오픈', '마음', '탕가', '감사', '진행', '운동', '소통', '문의', '전화', '서프', '야사', '근력', '시간', '피닝', '과정', '스포츠']


In [8]:
def yield_combined_df(exercise_name):
    file_name = "#" + exercise_name + "_sum.txt"
    file_1 = "/Users/noopy/FitCuration/" + file_name
    text = open(file_1, 'r', -1, "UTF-8", errors="ignore").read()
    # text

    clean_text = preprocessing_hangul(text)
    clean_text

    # MeCab으로 명사 뽑아내기
    mecab = Mecab()
    noun_list_mecab = mecab.nouns(clean_text)

    # stopwards preprocessing
    stopwords_mecab = ['수','퀄리티','도시','분','전문','스타','년','원',\
                       '월','화','수','목','금','시','앤','일','그램','문'] 
    stopwords_mecab += useless_list

    clean_noun_list_mecab = []

    for n in noun_list_mecab:
        if n not in stopwords_mecab:
            clean_noun_list_mecab.append(n)

    # get top 100 most common nouns
    nouns_mecab = Counter(clean_noun_list_mecab)
    tags_mecab = nouns_mecab.most_common(100)

    # Okt(Twitter 업데이트 버전)으로 서술어 뽑아내기

    twitter = Okt()

    # 4. 각 문장별로 형태소 구분하기
    morphed_list_okt = twitter.pos(clean_text)
    # morphed_list

    # 불용어
    stopwords_Twitter = ["입니다","있는","있습니다","같은","안녕하세요","고마워요","있어요","있게"\
                         ,"있도록","부탁드립니다","하는","합니다","할","하세요","하기","해","됩니다","하여",'잘','된','되고','되어','되었습니다',"없는","드립니다"\
                        ,"되기","하시는","하고","않을","같다","싶다","이런","저런","그런",'바랍니다'\
                        ,"했습니다","했다","해드립니다","하신","하실","않고","해요","가능합니다","하고싶으신"\
                        ,"않으며","주세요","오세요"]

    # 5. 형용사 품사만 뽑아내기
    adj_list_okt = []
    for word, tag in morphed_list_okt:
        if tag in ['Adjective','Verb'] and word not in stopwords_Twitter:
            adj_list_okt.append(word)

    # 6. 선별된 품사별 빈도수 계산 & 상위 빈도 10위 까지 출력
    adj_counts_okt = Counter(adj_list_okt)
    common_adj_okt = adj_counts_okt.most_common(100)
    # common_adj_okt
    noun_df = pd.DataFrame(np.sort(np.array(tags_mecab), axis=1), columns=[exercise_name+"명사개수",exercise_name+'관련명사'])
    adj_df = pd.DataFrame(np.sort(np.array(common_adj_okt), axis=1), columns=[exercise_name+ "서술어개수",exercise_name+'관련서술어'])
    combined_df = pd.concat([noun_df,adj_df],axis=1)
    return combined_df

In [2]:
exercise_to_loop.sort()
exercise_to_loop[:10]

['PT',
 '검도',
 '기구필라테스',
 '다빈치바디보드',
 '드럼스틱',
 '등산',
 '라틴댄스',
 '매트필라테스',
 '뮤직복싱',
 '바차타']

In [12]:
# test for Mecab & Okt function
%time df0 = yield_combined_df('PT')
df0

CPU times: user 2.86 s, sys: 49.9 ms, total: 2.91 s
Wall time: 2.8 s


,PT명사개수,PT관련명사,PT서술어개수,PT관련서술어
0,164,헬스,13,많은
1,102,다이어트,13,좋아요
2,83,필라테스,12,들
3,53,짐,9,해서
4,39,피티,9,좋은
...,...,...,...,...
95,8,주,2,쏘마
96,8,분당,2,힘들어도
97,8,스튜디오,2,하자
98,8,춘천,2,그러니


In [10]:
df0 = yield_combined_df('PT')

for item in exercise_to_loop:
    df0 = pd.concat([df0,yield_combined_df(item)],axis=1)
df0.to_csv("combined_nogada.csv", index=False, header=True)